# Setup for Amazon Bedrock
* Container: `conda_python3` <BR>
* We recommend `python 3.10` or later. 
    - version check: !python -V

## 0. Materials
- Bedrock user guide
    - https://docs.aws.amazon.com/bedrock/latest/userguide/what-is-service.html
- Step by step vidio tutorial
    - https://www.youtube.com/watch?v=ab1mbj0acDo

## 1. role setting (adding trust relationship)

### 1.1. role check

In [1]:
from sagemaker import get_execution_role

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/ec2-user/.config/sagemaker/config.yaml


In [2]:
strSageMakerRoleName = get_execution_role().rsplit('/', 1)[-1]
print (f"SageMaker Execution Role Name: {strSageMakerRoleName}")

SageMaker Execution Role Name: AmazonSageMaker-ExecutionRole-20240509T162476


### 1.2. policy
- 1.1에서 확인된 롤에 아래와 같이 4개의 권한 추가
    - AmazonBedrockFullAccess
    - AmazonOpenSearchServiceFullAccess
    - AmazonSSMFullAccess
    - AWSCloud9SSMAccessRole

## 2. Model access

### 2.1. Amazon Bedrock Console
![nn](../imgs/model-access/1.png)

### 2.2. "Get Started"
![nn](../imgs/model-access/2.png)

### 2.3. "Model access" - "Edit"
사용 모델 활성화 창
![nn](../imgs/model-access/3.png)

### 2.3. "Save Changes"
사용할 모델을 활성화 후 저장
![nn](../imgs/model-access/4.png)

## 3. Install python SDK for bedrock

- #### **Please change to `install_complex_pdf = True` when utilizing Complex PDF**.
    - ####  [Note] install_complex_pdf will take about 30 minutes.

In [3]:
install_needed = True
install_complex_pdf = True

In [ ]:
import os
import sys
import IPython
import subprocess

if install_needed:
    print("installing deps and restarting kernel")
    !{sys.executable} -m pip install -U pip
    !{sys.executable} -m pip install -U awscli
    !{sys.executable} -m pip install -U botocore
    !{sys.executable} -m pip install -U boto3
    !{sys.executable} -m pip install -U sagemaker 
    !{sys.executable} -m pip install -U langchain
    !{sys.executable} -m pip install -U langchain-community
    !{sys.executable} -m pip install -U langchain_aws
    !{sys.executable} -m pip install -U termcolor
    !{sys.executable} -m pip install -U transformers
    !{sys.executable} -m pip install -U librosa
    !{sys.executable} -m pip install -U opensearch-py
    !{sys.executable} -m pip install -U sqlalchemy #==2.0.1
    !{sys.executable} -m pip install -U pypdf
    #!{sys.executable} -m pip install -U spacy
    #!{sys.executable} -m spacy download ko_core_news_md
    !{sys.executable} -m pip install -U ipython
    !{sys.executable} -m pip install -U ipywidgets
    #!{sys.executable} -m pip install -U llmsherpa
    !{sys.executable} -m pip install -U anthropic
    !{sys.executable} -m pip install -U faiss-cpu
    !{sys.executable} -m pip install -U jq
    !{sys.executable} -m pip install -U pydantic

    if install_complex_pdf:

        response = subprocess.run(['cat', '/etc/os-release'], capture_output=True)
        response = response.stdout.decode("utf-8")

        if "Amazon Linux" in response: ## SageMaker Notebook

            !sudo rpm -Uvh https://dl.fedoraproject.org/pub/epel/epel-release-latest-7.noarch.rpm
            !sudo yum -y update
            !sudo yum install -y poppler-utils
            !{sys.executable} -m pip install -U lxml
            !{sys.executable} -m pip install -U kaleido
            !{sys.executable} -m pip install -U uvicorn
            !{sys.executable} -m pip install -U pandas
            !{sys.executable} -m pip install -U numexpr
            !{sys.executable} -m pip install -U pdf2image
            !sudo sh install_tesseract.sh
            !sudo amazon-linux-extras install libreoffice -y
            !{sys.executable} -m pip install -U "unstructured[all-docs]"
            !sudo rm -rf leptonica-1.84.1 leptonica-1.84.1.tar.gz tesseract-ocr

        else: ## SageMaker Studio
            !sudo apt-get install software-properties-common -y
            !sudo add-apt-repository ppa:alex-p/tesseract-ocr5 -y
            !sudo apt-get update -y
            !sudo apt-get install poppler-utils tesseract-ocr -y
            !sudo apt-get install libgl1-mesa-glx libglib2.0-0 -y
            !sudo apt install libreoffice -y
            !{sys.executable} -m pip install -U "unstructured[all-docs]"
            !{sys.executable} -m install -U pdf2image
            #!cd /usr/share/tesseract-ocr/5/tessdata/ && sudo wget https://github.com/tesseract-ocr/tessdata_best/raw/main/kor.traineddata

        ## Common
        !{sys.executable} -m pip install -U python-dotenv
        !{sys.executable} -m pip install -U llama-parse
        !{sys.executable} -m pip install -U pymupdf
        
    IPython.Application.instance().kernel.do_shutdown(True)

installing deps and restarting kernel
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 23.3 MB/s eta 0:00:00:00:01
  Attempting uninstall: pip
    Found existing installation: pip 23.3.2
    Uninstalling pip-23.3.2:
      Successfully uninstalled pip-23.3.2
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.5/4.5 MB 35.1 MB/s eta 0:00:00:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.2/12.2 MB 98.4 MB/s eta 0:00:00:00:010:01
  Attempting uninstall: botocore
    Found existing installation: botocore 1.34.93
    Uninstalling botocore-1.34.93:
      Successfully uninstalled botocore-1.34.93
  Attempting uninstall: awscli
    Found existing installation: awscli 1.32.93
    Uninstalling awscli-1.32.93:
      Successfully uninstalled awscli-1.32.93
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.3/139.3 kB 11.2 MB/s eta 0:00:00
  Attempting uninstall: boto3
    Found existing installation: boto3 1.34.93
    Uninstalling boto3-1.34.93:
      Successfully uninstalled boto3-1.34.93
   ━━

## 4. Check setting
모델 리스트 <BR>
![nn](../imgs/check.png)


In [3]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [4]:
import os
import sys
module_path = ".."
sys.path.append(os.path.abspath(module_path))

In [5]:
import boto3
import awscli
import botocore
import langchain
from pprint import pprint
from termcolor import colored
from utils.bedrock import bedrock_info

In [6]:
print(f"langchain version check: {langchain.__version__}")
print(f"boto3 version check: {boto3.__version__}")
print(f"botocore version check: {botocore.__version__}")
print(f"awscli version check: {awscli.__version__}")

langchain version check: 0.1.19
boto3 version check: 1.34.101
botocore version check: 1.34.101
awscli version check: 1.32.101


In [7]:
bedrock_region=boto3.Session().region_name

In [8]:
bedrock_client = boto3.client(
    service_name='bedrock-runtime',
    region_name=bedrock_region,
    endpoint_url=f"https://bedrock-runtime.{bedrock_region}.amazonaws.com"
)

In [9]:
print (colored("\n== FM lists ==", "green"))
pprint (bedrock_info.get_list_fm_models(verbose=False))


== FM lists ==
{'Claude-Instant-V1': 'anthropic.claude-instant-v1',
 'Claude-V1': 'anthropic.claude-v1',
 'Claude-V2': 'anthropic.claude-v2',
 'Claude-V2-1': 'anthropic.claude-v2:1',
 'Claude-V3-Haiku': 'anthropic.claude-3-haiku-20240307-v1:0',
 'Claude-V3-Sonnet': 'anthropic.claude-3-sonnet-20240229-v1:0',
 'Cohere-Embeddings-En': 'cohere.embed-english-v3',
 'Cohere-Embeddings-Multilingual': 'cohere.embed-multilingual-v3',
 'Command': 'cohere.command-text-v14',
 'Command-Light': 'cohere.command-light-text-v14',
 'Jurassic-2-Mid': 'ai21.j2-mid-v1',
 'Jurassic-2-Ultra': 'ai21.j2-ultra-v1',
 'Llama2-13b-Chat': 'meta.llama2-13b-chat-v1',
 'Titan-Embeddings-G1': 'amazon.titan-embed-text-v1',
 'Titan-Text-G1': 'amazon.titan-text-express-v1',
 'Titan-Text-G1-Light': 'amazon.titan-text-lite-v1'}


## 5. generation

In [12]:
import base64
from langchain_aws import ChatBedrock, BedrockLLM
from langchain.schema.output_parser import StrOutputParser
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler
from langchain_core.prompts import ChatPromptTemplate, HumanMessagePromptTemplate, SystemMessagePromptTemplate

### Titan

In [19]:
llm = BedrockLLM(
    model_id=bedrock_info.get_model_id(model_name="Titan-Text-G1"),
    client=bedrock_client,
    model_kwargs={
        "maxTokenCount":512,
        "stopSequences":[],
        "temperature":0,
        "topP":0.9
    },
    streaming=True,
    callbacks=[StreamingStdOutCallbackHandler()]
)

In [20]:
prompt = "Please let us know SageMaker's advantages in 100 words"

In [21]:
print (colored(llm(prompt), "green"))


SageMaker is a fully managed machine learning service that enables you to build, train, and deploy high-quality machine learning models quickly. It provides a wide range of capabilities, including natural language processing, computer vision, and time series analysis, and supports various programming languages and frameworks, such as Python, R, and Java.

One of the key advantages of SageMaker is its ease of use. It provides a user-friendly interface that allows you to drag and drop data, select pre-trained models, and configure your machine learning experiments without writing any code. This makes it accessible to a wide range of users, including data scientists, engineers, and business professionals, who may not have extensive experience with machine learning.

Another advantage of SageMaker is its scalability. It can handle large datasets and complex models, making it ideal for deploying machine learning solutions at scale. SageMaker also provides built-in features for monitoring a

### Claude v3

In [23]:
llm = ChatBedrock(
    model_id=bedrock_info.get_model_id(model_name="Claude-V3-Sonnet"),
    client=bedrock_client,
    model_kwargs={
        "max_tokens": 1024,
        "temperature":0,
        "top_p":0.9,
        "stop_sequences": ["\n\nHuman"],
    },
    streaming=True,
    callbacks=[StreamingStdOutCallbackHandler()],
)
llm

ChatBedrock(callbacks=[<langchain_core.callbacks.streaming_stdout.StreamingStdOutCallbackHandler object at 0x7f675be12230>], client=<botocore.client.BedrockRuntime object at 0x7f672fb8eb00>, model_id='anthropic.claude-3-sonnet-20240229-v1:0', model_kwargs={'max_tokens': 1024, 'temperature': 0, 'top_p': 0.9, 'stop_sequences': ['\n\nHuman']}, streaming=True)

In [24]:
system_prompt = '''
                You are a master answer bot designed to answer user's questions.
                I'm going to ask you a question.
                '''
system_message_template = SystemMessagePromptTemplate.from_template(system_prompt)

human_prompt = [
    {
        "type": "text",
        "text": '''
                Here is the question: <question>{question}</question>

                Answer in Korean.
                If the question cannot be answered by the contexts, say "No relevant contexts".
                
        '''
    }           
]
human_message_template = HumanMessagePromptTemplate.from_template(human_prompt)

prompt = ChatPromptTemplate.from_messages(
    [system_message_template, human_message_template]
)

chain = prompt | llm | StrOutputParser()

In [25]:
response = chain.invoke(
    {
        "question": "세이지메이커의 장점을 100단어로 알려주세요."
    }
)

세이지메이커의 주요 장점은 다음과 같습니다:

1. 강력한 데이터 처리 및 분석 기능을 갖추고 있어 대규모 데이터를 효율적으로 다룰 수 있습니다.
2. 다양한 통계 및 기계 학습 알고리즘을 제공하여 복잡한 분석 작업을 수행할 수 있습니다. 
3. 직관적인 사용자 인터페이스로 코딩 없이도 데이터 분석 파이프라인을 구축할 수 있습니다.
4. 클라우드 기반으로 확장성이 뛰어나며, 대규모 컴퓨팅 리소스에 쉽게 접근할 수 있습니다.
5. 다양한 데이터 소스와 통합되어 데이터 준비 및 변환 작업을 간소화합니다.

### Claude v3 sonnet and haiku
## test: 아래 이미지에 대한 질문
![nn](../imgs/setup/attention-is-all-you-needs.png)

In [26]:
llm = ChatBedrock(
    model_id=bedrock_info.get_model_id(model_name="Claude-V3-Haiku"), # or "Claude-V3-Sonnet"
    client=bedrock_client,
    streaming=True,
    callbacks=[StreamingStdOutCallbackHandler()],
    model_kwargs={
        "max_tokens": 1024,
        "stop_sequences": ["\n\nHuman"],
        # "temperature": 0,
        # "top_k": 350,
        # "top_p": 0.999
    }
)
llm

ChatBedrock(callbacks=[<langchain_core.callbacks.streaming_stdout.StreamingStdOutCallbackHandler object at 0x7f6748365150>], client=<botocore.client.BedrockRuntime object at 0x7f672fb8eb00>, model_id='anthropic.claude-3-haiku-20240307-v1:0', model_kwargs={'max_tokens': 1024, 'stop_sequences': ['\n\nHuman']}, streaming=True)

In [27]:
system_prompt = '''
                You are a master answer bot designed to answer user's questions.
                I'm going to give you contexts which consist of texts and images.
                Read the contexts carefully, because I'm going to ask you a question about it.
                '''
system_message_template = SystemMessagePromptTemplate.from_template(system_prompt)

In [28]:
with open("../imgs/setup/attention-is-all-you-needs.png", "rb") as image_file:
    encoded_string = base64.b64encode(image_file.read())
    base64_string = encoded_string.decode('utf-8')
    
human_prompt = [
    {
        "type": "image_url",
        "image_url": {
            "url": f"data:image/png;base64,{base64_string}",
        },
    },
    {
        "type": "text",
        "text": '''
                Here is the question: <question>{question}</question>

                Answer in Korean.
                If the question cannot be answered by the contexts, say "No relevant contexts".
                
        '''
    }           
]
human_message_template = HumanMessagePromptTemplate.from_template(human_prompt)

In [29]:
prompt = ChatPromptTemplate.from_messages(
    [system_message_template, human_message_template]
)

chain = prompt | llm | StrOutputParser()

response = chain.invoke(
    {
        "question": "MoE 모델의 EN-FR BLEU 스코어는 무엇입니까?"
    }
)

모E (MoE) 모델의 EN-FR BLEU 스코어는 40.56입니다.